## Load Packages

In [1]:
include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Mean-Variance: Brute Force

MV analysis starts with inputs of the expected returns and their covariance matrix.

In [3]:
μ = [0.115; 0.095; 0.06]        #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2                  

println("expected returns: ")
printmat(μ)
println("covariance matrix:")
printmat(Σ)

expected returns: 
     0.115
     0.095
     0.060

covariance matrix:
     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010



To calculate the expeted return and the variance of a portfolio, use $ER_p = w'\mu$ and $Var(R_p) = w'\Sigma w$.

In [4]:
w = [ 0.00  0.22  0.02  0.25;  #4 different portfolios (try many)
      1.00  0.30  0.63  0.68;
      0.00  0.48  0.35  0.07]

println("Several different portfolios (the weights of each is in a column): ")
printmat(w)

K = size(w,2)
ERp   = fill(NaN,K)
StdRp = fill(NaN,K)
for i = 1:K             #calculate ER and StdR for each of the k different portfolios 
    ERp[i]   = (w[:,i]'μ)[1]
    StdRp[i] = sqrt((w[:,i]'Σ*w[:,i])[1])
end    
println("expected portfolio return and Std of portfolio return for $K portfolios, %: ")
printmat([ERp StdRp]*100)

Several different portfolios (the weights of each is in a column): 
     0.000     0.220     0.020     0.250
     1.000     0.300     0.630     0.680
     0.000     0.480     0.350     0.070

expected portfolio return and Std of portfolio return for 4 portfolios, %: 
     9.500     8.000
     8.260     7.398
     8.315     6.415
     9.755     7.378



In [5]:
plot1 = scatter(StdRp*100,ERp*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, %

# MV Frontier with Two Assets

Recall: with only two investable assets, all portfolios of them are on the MV frontier

In [6]:
μ = [0.115; 0.06]            #means
Σ  = [166   58;              #covariance matrix
       58  100]/100^2                      

w1 = collect(1:-0.05:0)            #weight on asset 1, only using w1>0 
n  = length(w1)
ERp   = fill(NaN,n)
StdRp = fill(NaN,n)
for i = 1:n
    w        = [w1[i];1-w1[i]]
    ERp[i]   = (w'μ)[1]
    StdRp[i] = sqrt((w'Σ*w)[1])
end   

plot2 = plot(StdRp*100,ERp*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, %

# Calculating the MV Frontier (of only Risky Assets)

All portfolio on the MV frontier of risky assets only have (vectors of) portfolio weights as in

$w = \Sigma^{-1}(\mu \lambda + \mathbf{1} \delta)$, 

where $\lambda$ and $\delta$ depends on $(\mu,\Sigma)$ and the required average return $\mu^*$

In [7]:
μ = [0.115; 0.095; 0.06]    #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2                  
Rf = 0.03

println("μ, Σ and Rf: ")
printmat(μ)
printmat(Σ)
printmat(Rf)

μ, Σ and Rf: 
     0.115
     0.095
     0.060

     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010

     0.030



Some of the intermediate calculations are as follows. (Later on we put it all in a function)

In [8]:
mustar = 0.1                          #required average return  

n    = length(μ)
oneV = ones(n,1) 
 
Σ_1  = inv(Σ)

println("inv(Σ):")
printmat(Σ_1)
println("Σ*inv(Σ):")
printmat(Σ*Σ_1)

A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
B    = (μ'Σ_1*oneV)[1]
C    = (oneV'Σ_1*oneV)[1]
printlnPs("A, B, C: ",[A B C ])

λ    = (C*mustar - B)/(A*C-B^2)
δ    = (A-B*mustar)/(A*C-B^2)
printlnPs("λ and δ: ",[λ δ])

w    = Σ_1 *(μ*λ + oneV*δ)
println("w: ")
printmat(w)

inv(Σ):
    85.735   -42.545   -48.024
   -42.545   177.754    17.566
   -48.024    17.566   127.151

Σ*inv(Σ):
     1.000     0.000     0.000
     0.000     1.000     0.000
    -0.000     0.000     1.000

A, B, C:      1.804    19.759   244.634
λ and δ:      0.093    -0.003
w: 
     0.288
     0.690
     0.022



In [9]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF of risky assets
    n    = length(μ)
    oneV = ones(n,1) 
    Σ_1  = inv(Σ)
    A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
    B    = (μ'Σ_1*oneV)[1]
    C    = (oneV'Σ_1*oneV)[1]
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalc (generic function with 1 method)

In [10]:
(StdAt10,wAt10) = MVCalc(0.1,μ,Σ)
println("Just testing: std and w of portfolio with mean return 10: ", round(StdAt10,3))
printmat(wAt10)

Just testing: std and w of portfolio with mean return 10: 0.077
     0.288
     0.690
     0.022



In [11]:
mustar = linspace(Rf,0.15,101)
L      = length(mustar)
StdRp  = fill(NaN,L)
for i = 1:L
    StdRP_i, = MVCalc(mustar[i],μ,Σ)             #, = means skipping the remaining output
    StdRp[i] = StdRP_i
end    

plot3 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 514.913,303.465 506.763,300.706 498.663,297.946 490.617,295.187 482.628,292.428 474.697,289.669 466.83,286.91 459.03,284.151 451.3,281.392 443.645,278.633 
 436.07,275.874 428.579,273.115 421.177,270.356 413.87,267.597 406.663,264.838 399.564,262.079 392.579,259.32 385.715,256.561 378.98,253.802 372.382,251.043 
 365.93,248.283 359.633,245.524 353.501,242.765 347.544,240.006 341.773,237.247 336.2,234.488 330.836,231.729 325.694,228.97 320.788,226.211 316.13,223.452 
 311.733,220.693 307.612,217.934 303.78,215.175 300.251,212.416 297.037,209.657 294.152,206.898 291.608,204.139 289.416,201.38 287.586,198.62 286.126,195.861 
 285.043,193.102 284.343,190.343 284.029,187.584 284.103,184.825 284.565,182.066 285.412,179.307 286.639,176.548 288.242,173.789 290.212,171.03 292.539,168.271 
 295.215,165.512 298.227,162.753 301.562,159.994 305.209,157.235 309.153,154.476 313.381,151.717 317.879,148.957 322.634,146.198 327.632,143.439 332.86,140.68 
 338.305,137.921 343.955,135.162 349.798,132.403 355.824,129.644 362.02,126.885 368.378,124.126 374.887,121.367 381.538,118.608 388.323,115.849 395.234,113.09 
 402.264,110.331 409.404,107.572 416.65,104.813 423.994,102.054 431.431,99.2945 438.955,96.5354 446.561,93.7764 454.245,91.0173 462.002,88.2583 469.829,85.4992 
 477.72,82.7402 485.674,79.9811 493.685,77.222 501.752,74.463 509.871,71.7039 518.04,68.9449 526.255,66.1858 534.515,63.4268 542.818,60.6677 551.16,57.9087 
 559.541,55.1496 567.957,52.3906 576.409,49.6315 584.893,46.8724 593.409,44.1134 601.954,41.3543 610.528,38.5953 619.129,35.8362 627.756,33.0772 636.407,30.3181 
 645.083,27.5591 
 "/>

# Calculating the MV Frontier (of Risky and Riskfree Assets)

All portfolios on the MV frontier of both risky and riskfree have (vectors of) portfolio weights on the risky assets as in 

$w=\frac{\mu^{\ast}-R_{f}}{(\mu^{e})^{\prime}\Sigma^{-1}\mu^{e}}\Sigma^{-1}
\mu^{e}$, 

where $\mu^*$ is the required average return.

The weight of the riskfree asset is $1-\mathbf{1}'w$

In [12]:
function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF of (Risky,Riskfree)
    n     = length(μ)
    μe    = μ - Rf                         #expected excess returns            
    Σ_1   = inv(Σ)
    w     = (mustar-Rf)/(μe'Σ_1*μe)[1] * Σ_1 *μe
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

In [13]:
StdRpRf  = fill(NaN,L)
for i = 1:L
    StdRpRf_i, = MVCalcRf(mustar[i],μ,Σ,Rf) 
    StdRpRf[i] = StdRpRf_i
end    

plot4 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 514.913,303.465 506.763,300.706 498.663,297.946 490.617,295.187 482.628,292.428 474.697,289.669 466.83,286.91 459.03,284.151 451.3,281.392 443.645,278.633 
 436.07,275.874 428.579,273.115 421.177,270.356 413.87,267.597 406.663,264.838 399.564,262.079 392.579,259.32 385.715,256.561 378.98,253.802 372.382,251.043 
 365.93,248.283 359.633,245.524 353.501,242.765 347.544,240.006 341.773,237.247 336.2,234.488 330.836,231.729 325.694,228.97 320.788,226.211 316.13,223.452 
 311.733,220.693 307.612,217.934 303.78,215.175 300.251,212.416 297.037,209.657 294.152,206.898 291.608,204.139 289.416,201.38 287.586,198.62 286.126,195.861 
 285.043,193.102 284.343,190.343 284.029,187.584 284.103,184.825 284.565,182.066 285.412,179.307 286.639,176.548 288.242,173.789 290.212,171.03 292.539,168.271 
 295.215,165.512 298.227,162.753 301.562,159.994 305.209,157.235 309.153,154.476 313.381,151.717 317.879,148.957 322.634,146.198 327.632,143.439 332.86,140.68 
 338.305,137.921 343.955,135.162 349.798,132.403 355.824,129.644 362.02,126.885 368.378,124.126 374.887,121.367 381.538,118.608 388.323,115.849 395.234,113.09 
 402.264,110.331 409.404,107.572 416.65,104.813 423.994,102.054 431.431,99.2945 438.955,96.5354 446.561,93.7764 454.245,91.0173 462.002,88.2583 469.829,85.4992 
 477.72,82.7402 485.674,79.9811 493.685,77.222 501.752,74.463 509.871,71.7039 518.04,68.9449 526.255,66.1858 534.515,63.4268 542.818,60.6677 551.16,57.9087 
 559.541,55.1496 567.957,52.3906 576.409,49.6315 584.893,46.8724 593.409,44.1134 601.954,41.3543 610.528,38.5953 619.129,35.8362 627.756,33.0772 636.407,30.3181 
 645.083,27.5591 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.1181,303.465 59.8101,300.706 64.5022,297.946 69.1942,295.187 73.8862,292.428 78.5782,289.669 83.2703,286.91 87.9623,284.151 92.6543,281.392 97.3463,278.633 
 102.038,275.874 106.73,273.115 111.422,270.356 116.114,267.597 120.806,264.838 125.498,262.079 130.19,259.32 134.883,256.561 139.575,253.802 144.267,251.043 
 148.959,248.283 153.651,245.524 158.343,242.765 163.035,240.006 167.727,237.247 172.419,234.488 177.111,231.729 181.803,228.97 186.495,226.211 191.187,223.452 
 195.879,220.693 200.571,217.934 205.263,215.175 209.955,212.416 214.647,209.657 219.339,206.898 224.031,204.139 228.723,201.38 233.415,198.62 238.107,195.861 
 242.799,193.102 247.491,190.343 252.183,187.584 256.875,184.825 261.567,182.066 266.259,179.307 270.951,176.548 275.643,173.789 280.335,171.03 285.027,168.271 
 289.719,165.512 294.411,162.753 299.103,159.994 303.795,157.235 308.487,154.476 313.179,151.717 317.871,148.957 322.563,146.198 327.256,143.439 331.948,140.68 
 336.64,137.921 341.332,135.162 346.024,132.403 350.716,129.644 355.408,126.885 360.1,124.126 364.792,121.367 369.484,118.608 374.176,115.849 378.868,113.09 
 383.56,110.331 388.252,107.572 392.944,104.813 397.636,102.054 402.328,99.2945 407.02,96.5354 411.712,93.7764 416.404,91.0173 421.096,88.2583 425.788,85.4992 
 430.48,82.7402 435.172,79.9811 439.864,77.222 444.556,74.463 449.248,71.7039 453.94,68.9449 458.632,66.1858 463.324,63.4268 468.016,60.6677 472.708,57.9087 
 477.4,55.1496 482.092,52.3906 486.784,49.6315 491.476,46.8724 496.168,44.1134 500.86,41.3543 505.552,38.5953 510.244,35.8362 514.936,33.0772 519.628,30.3181 
 524.321,27.5591 
 "/>

# Tangency Portfolio

The tangency portfolio is a particular portfolio on the MV frontier of risky and riskfree, where the weights on the risky assets sum to one. It is therefore also on the MV frontier of risky assets only. The vector of portfolio weights is

$w_{T}=\frac{\Sigma^{-1}\mu^{e}}{\mathbf{1}^{\prime}\Sigma^{-1}\mu^{e}}$

In [14]:
function MVTangencyP(μ,Σ,Rf)           #calculates the tangency portfolio
    n     = length(μ)
    oneV  = ones(n,1)  
    μe    = μ - Rf                     #expected excess returns            
    Σ_1   = inv(Σ)
    w     = Σ_1 *μe/(oneV'Σ_1*μe)[1]
    muT   = (w'μ + (1-sum(w))*Rf)[1]
    StdT = sqrt((w'Σ*w)[1])
    return w,muT,StdT
end

MVTangencyP (generic function with 1 method)

In [15]:
(wT,muT,StdT) = MVTangencyP(μ,Σ,Rf)
println("Tangency portfolio: ")
printmat(wT)
printlnPs("mean and std of tangency portfolio, %: ",[muT StdT]*100)

Tangency portfolio: 
     0.248
     0.682
     0.070

mean and std of tangency portfolio, %:      9.750     7.372


By mixing the tangency portfolio and the riskfree, we can create any point on the MV frontier (for risky and riskfree). 

The code below shows the expected return and standard deviation of several portfolio (different $v$ values) of the form

$R_p = v w_T'R + (1-v)R_f$ 

In [16]:
v = [0;0.44;1;1.41]                                 #try different mixes of wT and Rf

(ERLev,StdLev)  = (similar(v),similar(v))           #to store results in
for i = 1:length(v)
    w          = v[i]*wT                          #risky portfolio
    ERLev[i]   = (w'μ + (1-v[i])*Rf)[1]     #portfolio with v[i] in wT and 1-v[i] in Rf
    StdLev[i]  = sqrt((w'Σ*w)[1])
end


plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(StdLev*100,ERLev*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 514.913,303.465 506.763,300.706 498.663,297.946 490.617,295.187 482.628,292.428 474.697,289.669 466.83,286.91 459.03,284.151 451.3,281.392 443.645,278.633 
 436.07,275.874 428.579,273.115 421.177,270.356 413.87,267.597 406.663,264.838 399.564,262.079 392.579,259.32 385.715,256.561 378.98,253.802 372.382,251.043 
 365.93,248.283 359.633,245.524 353.501,242.765 347.544,240.006 341.773,237.247 336.2,234.488 330.836,231.729 325.694,228.97 320.788,226.211 316.13,223.452 
 311.733,220.693 307.612,217.934 303.78,215.175 300.251,212.416 297.037,209.657 294.152,206.898 291.608,204.139 289.416,201.38 287.586,198.62 286.126,195.861 
 285.043,193.102 284.343,190.343 284.029,187.584 284.103,184.825 284.565,182.066 285.412,179.307 286.639,176.548 288.242,173.789 290.212,171.03 292.539,168.271 
 295.215,165.512 298.227,162.753 301.562,159.994 305.209,157.235 309.153,154.476 313.381,151.717 317.879,148.957 322.634,146.198 327.632,143.439 332.86,140.68 
 338.305,137.921 343.955,135.162 349.798,132.403 355.824,129.644 362.02,126.885 368.378,124.126 374.887,121.367 381.538,118.608 388.323,115.849 395.234,113.09 
 402.264,110.331 409.404,107.572 416.65,104.813 423.994,102.054 431.431,99.2945 438.955,96.5354 446.561,93.7764 454.245,91.0173 462.002,88.2583 469.829,85.4992 
 477.72,82.7402 485.674,79.9811 493.685,77.222 501.752,74.463 509.871,71.7039 518.04,68.9449 526.255,66.1858 534.515,63.4268 542.818,60.6677 551.16,57.9087 
 559.541,55.1496 567.957,52.3906 576.409,49.6315 584.893,46.8724 593.409,44.1134 601.954,41.3543 610.528,38.5953 619.129,35.8362 627.756,33.0772 636.407,30.3181 
 645.083,27.5591 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.1181,303.465 59.8101,300.706 64.5022,297.946 69.1942,295.187 73.8862,292.428 78.5782,289.669 83.2703,286.91 87.9623,284.151 92.6543,281.392 97.3463,278.633 
 102.038,275.874 106.73,273.115 111.422,270.356 116.114,267.597 120.806,264.838 125.498,262.079 130.19,259.32 134.883,256.561 139.575,253.802 144.267,251.043 
 148.959,248.283 153.651,245.524 158.343,242.765 163.035,240.006 167.727,237.247 172.419,234.488 177.111,231.729 181.803,228.97 186.495,226.211 191.187,223.452 
 195.879,220.693 200.571,217.934 205.263,215.175 209.955,212.416 214.647,209.657 219.339,206.898 224.031,204.139 228.723,201.38 233.415,198.62 238.107,195.861 
 242.799,193.102 247.491,190.343 252.183,187.584 256.875,184.825 261.567,182.066 266.259,179.307 270.951,176.548 275.643,173.789 280.335,171.03 285.027,168.271 
 289.719,165.512 294.411,162.753 299.103,159.994 303.795,157.235 308.487,154.476 313.179,151.717 317.871,148.957 322.563,146.198 327.256,143.439 331.948,140.68 
 336.64,137.921 341.332,135.162 346.024,132.403 350.716,129.644 355.408,126.885 360.1,124.126 364.792,121.367 369.484,118.608 374.176,115.849 378.868,113.09 
 383.56,110.331 388.252,107.572 392.944,104.813 397.636,102.054 402.328,99.2945 407.02,96.5354 411.712,93.7764 416.404,91.0173 421.096,88.2583 425.788,85.4992 
 430.48,82.7402 435.172,79.9811 439.864,77.222 444.556,74.463 449.248,71.7039 453.94,68.9449 458.632,66.1858 463.324,63.4268 468.016,60.6677 472.708,57.9087 
 477.4,55.1496 482.092,52.3906 486.784,49.6315 491.476,46.8724 496.168,44.1134 500.86,41.3543 505.552,38.5953 510.244,35.8362 514.936,33.0772 519.628,30.3181 
 524.321,27.5591 
 "/>

## Examples of Tangency Portfolios

In [17]:
μb = [9; 6]/100                     #means
Σb = [ 256  0;
      0    144]/100^2
Rfb = 1/100
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

wT, = MVTangencyP([13; 6]/100,Σb,Rfb)
printmat(wT)

Σb = [ 1  -0.8;
      -0.8    1]
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

Σb = [ 1  0.8;
      0.8    1]
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

     0.474
     0.526

     0.574
     0.426

     0.513
     0.487

     1.538
    -0.538



# Drawing the MV Frontier Revisisted (extra)

Recall: with only two investable assets, all portfolios of them are on the MV frontier.

We apply this idea by using the global minimum variance portfolio (see below for code) and the tangency portfolios (see above).

In [18]:
function MVMinimumVarP(μ,Σ,Rf)           #calculates the global minimum variance portfolio
    n     = length(μ)
    oneV  = ones(n,1)  
    μe    = μ - Rf                     
    Σ_1   = inv(Σ)
    w     = Σ_1*oneV/(oneV'Σ_1*oneV)[1]
    mu   = (w'μ + (1-sum(w))*Rf)[1]
    Std = sqrt((w'Σ*w)[1])
    return w,mu,Std
end

MVMinimumVarP (generic function with 1 method)

In [19]:
(wT,muT,StdT) = MVTangencyP(μ,Σ,Rf)
(wMvp,muMvp,StdMvp) = MVMinimumVarP(μ,Σ,Rf)
println("Tangency and minum variance portfolios: ")
printmat([wT wMvp])

Tangency and minum variance portfolios: 
     0.248    -0.020
     0.682     0.625
     0.070     0.395



In [20]:
v = linspace(-2.5,2.5,101)                                 #try different mixes of wT and wMvp

(ERLev,StdLev)  = (similar(v),similar(v))
for i = 1:length(v)
    w          = v[i]*wT + (1-v[i])*wMvp                        #portfolio with v[i] in wT and 1-v[i] in wMvp
    ERLev[i]   = (w'μ)[1]
    StdLev[i]  = sqrt((w'Σ*w)[1])
end


plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(StdLev*100,ERLev*100)
title!("Mean vs standard deviation: scatter points are new")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation: scatter points are new 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 514.913,303.465 506.763,300.706 498.663,297.946 490.617,295.187 482.628,292.428 474.697,289.669 466.83,286.91 459.03,284.151 451.3,281.392 443.645,278.633 
 436.07,275.874 428.579,273.115 421.177,270.356 413.87,267.597 406.663,264.838 399.564,262.079 392.579,259.32 385.715,256.561 378.98,253.802 372.382,251.043 
 365.93,248.283 359.633,245.524 353.501,242.765 347.544,240.006 341.773,237.247 336.2,234.488 330.836,231.729 325.694,228.97 320.788,226.211 316.13,223.452 
 311.733,220.693 307.612,217.934 303.78,215.175 300.251,212.416 297.037,209.657 294.152,206.898 291.608,204.139 289.416,201.38 287.586,198.62 286.126,195.861 
 285.043,193.102 284.343,190.343 284.029,187.584 284.103,184.825 284.565,182.066 285.412,179.307 286.639,176.548 288.242,173.789 290.212,171.03 292.539,168.271 
 295.215,165.512 298.227,162.753 301.562,159.994 305.209,157.235 309.153,154.476 313.381,151.717 317.879,148.957 322.634,146.198 327.632,143.439 332.86,140.68 
 338.305,137.921 343.955,135.162 349.798,132.403 355.824,129.644 362.02,126.885 368.378,124.126 374.887,121.367 381.538,118.608 388.323,115.849 395.234,113.09 
 402.264,110.331 409.404,107.572 416.65,104.813 423.994,102.054 431.431,99.2945 438.955,96.5354 446.561,93.7764 454.245,91.0173 462.002,88.2583 469.829,85.4992 
 477.72,82.7402 485.674,79.9811 493.685,77.222 501.752,74.463 509.871,71.7039 518.04,68.9449 526.255,66.1858 534.515,63.4268 542.818,60.6677 551.16,57.9087 
 559.541,55.1496 567.957,52.3906 576.409,49.6315 584.893,46.8724 593.409,44.1134 601.954,41.3543 610.528,38.5953 619.129,35.8362 627.756,33.0772 636.407,30.3181 
 645.083,27.5591 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.1181,303.465 59.8101,300.706 64.5022,297.946 69.1942,295.187 73.8862,292.428 78.5782,289.669 83.2703,286.91 87.9623,284.151 92.6543,281.392 97.3463,278.633 
 102.038,275.874 106.73,273.115 111.422,270.356 116.114,267.597 120.806,264.838 125.498,262.079 130.19,259.32 134.883,256.561 139.575,253.802 144.267,251.043 
 148.959,248.283 153.651,245.524 158.343,242.765 163.035,240.006 167.727,237.247 172.419,234.488 177.111,231.729 181.803,228.97 186.495,226.211 191.187,223.452 
 195.879,220.693 200.571,217.934 205.263,215.175 209.955,212.416 214.647,209.657 219.339,206.898 224.031,204.139 228.723,201.38 233.415,198.62 238.107,195.861 
 242.799,193.102 247.491,190.343 252.183,187.584 256.875,184.825 261.567,182.066 266.259,179.307 270.951,176.548 275.643,173.789 280.335,171.03 285.027,168.271 
 289.719,165.512 294.411,162.753 299.103,159.994 303.795,157.235 308.487,154.476 313.179,151.717 317.871,148.957 322.563,146.198 327.256,143.439 331.948,140.68 
 336.64,137.921 341.332,135.162 346.024,132.403 350.716,129.644 355.408,126.885 360.1,124.126 364.792,121.367 369.484,118.608 374.176,115.849 378.868,113.09 
 383.56,110.331 388.252,107.572 392.944,104.813 397.636,102.054 402.328,99.2945 407.02,96.5354 411.712,93.7764 416.404,91.0173 421.096,88.2583 425.788,85.4992 
 430.48,82.7402 435.172,79.9811 439.864,77.222 444.556,74.463 449.248,71.7039 453.94,68.9449 458.632,66.1858 463.324,63.4268 468.016,60.6677 472.708,57.9087 
 477.4,55.1496 482.092,52.3906 486.784,49.6315 491.476,46.8724 496.168,44.1134 500.86,41.3543 505.552,38.5953 510.244,35.8362 514.936,33.0772 519.628,30.3181 
 524.321,27.5591 
 "/>